In [ ]:
# import mantid algorithms, numpy and matplotlib
import matplotlib.pyplot as plt
plt.rcParams['figure.figsize'] = [10, 8]
#%matplotlib notebook

from mantid.simpleapi import *

import numpy as np
from mantid import plots
from mantid.simpleapi import CreateWorkspace
import spline_background
import scipp as sc
from scipp import Dim

## DAS-145 Demo
This notebook shows behaviour of the background spline method implemented in scipp/ess. A random, noisy dataset is generated from a gaussian distribution. A second non-random dataset is generated as well. Splines for both cases are generated with scipp and Mantid and compared and plotted.

In [ ]:
# Create a workspace with Gaussian peak
x = np.arange(30)
y0 = 20.+50.*np.exp(-(x-8.)**2./120.)
err = np.sqrt(y0)
y = 20.+50.*np.exp(-(x-8.)**2./120.)
y += err*np.random.normal(size=len(err))
err = np.sqrt(y)
var_x = sc.Variable(dims=[sc.Dim.X], values=x)
var_y = sc.Variable(dims=[sc.Dim.X], values=y, variances=err**2)
da_1 = sc.DataArray(data=var_y, coords={sc.Dim.X: var_x})

# Create Mantid workspaces for easy plotting
w_1 = CreateWorkspace(DataX=x, DataY=y, DataE=err, NSpec=1, UnitX='tof')

In [ ]:
# Create a damped sinusoidal example
N = 100
x_2 = np.arange(N)
y0 = np.zeros(N)
y_2 = y0
for i, item in enumerate(y_2):
    xi = (15.0 / (N-1)) * i
    y_2[i] = np.cos(xi) * np.exp(-0.1*xi)
err_2 = np.sqrt(y_2*y_2)

var_x = sc.Variable(dims=[sc.Dim.X], values=x_2)
var_y = sc.Variable(dims=[sc.Dim.X], values=y_2, variances=err_2**2)
da_2 = sc.DataArray(data=var_y, coords={sc.Dim.X: var_x})

# Create Mantid workspaces for easy plotting
w_2 = CreateWorkspace(DataX=x_2, DataY=y_2, DataE=err_2, NSpec=1, UnitX='tof')    

In [ ]:
def plot_mantid_spline(w, x, y, err):
    fig, ax = plt.subplots(subplot_kw={'projection':'mantid'})
    ax.plot(w,'x', label='Original data') # plot the initial distribution with black line

    spline = SplineBackground(w_1)
    ax.plot(spline, label='Splined data')
    ax.legend(['Generated data', 'Spline'])

### Reference Mantid plot, SplineBackground method

In [ ]:
plot_mantid_spline(w_1, x, y0, err)

### Comparison between Mantid SplineBackground and scipp spline_background methods

In [ ]:
def plot_comparison(w, da, x, y):
    fig, ax = plt.subplots(subplot_kw={'projection':'mantid'})
    ax.plot(x, y,'x', label='Base distribution') # plot the initial distribution with black line
    
    output_array = spline_background.bspline_background(da, Dim.X)
    x_sp = output_array.coords[sc.Dim.X].values
    y_sp = output_array.values    
    ax.plot(x_sp, y_sp,'b-', label='scipp spline') # plot the scipp spline
    
    output_mantid = SplineBackground(w, NCoeff=20)
    ax.plot(output_mantid, 'c-', label='mantid spline') # plot the mantid spline
    ax.legend(['Original distribution', 'Scipp spline', 'Mantid spline'])

In [ ]:
plot_comparison(w_1, da_1, x, y)

In [ ]:
plot_comparison(w_2, da_2, x_2, y_2)